# Inport needed package

- import os, sys # to add the parent directory to the path

In [1]:
import os
import sys
import time

- Using torchvision to create a dataset

In [2]:
# Using torchvision to create a dataset
import cv2
from torchvision import transforms
import torch
from torch.utils.data import random_split, DataLoader
import torchvision

import pandas as pd

- import self library

In [3]:
from train.trainer import ClassifierTrainer as Trainer
import dataset as ds  # type: ignore
import model as md  # type: ignore

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Datasets

In [26]:
import torchvision
import dataset as ds  # type: ignore
import pandas as pd

## Nus2Hands dataset


In [27]:
nus_2_hand_sign_ds = ds.ImageDataset(
    './datasets/nus2hands',
    file_extension='jpg',
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor()
    ]),
)
hgr1_ds_list = [ds.ImageDataset(
    f'datasets\HGR1\id{i:02d}',
    file_extension='jpg',
    transform=torchvision.transforms.Compose([
        torchvision.transforms.ToTensor()
    ]),
) for i in range(1, 13)]
fer2013_ds_list = [
    ds.ImageDataset(
        './datasets/PER-2013/train',
        file_extension='jpg',
        transform=torchvision.transforms.Compose([
            torchvision.transforms.ToTensor()
        ]),
    ),
    ds.ImageDataset(
        './datasets/PER-2013/test',
        file_extension='jpg',
        transform=torchvision.transforms.Compose([
            torchvision.transforms.ToTensor()
        ]),
    )
]
print(f'Number of samples in Nus2Hands dataset: {len(nus_2_hand_sign_ds)}')
print(f'Number of samples in HGR1 dataset: {sum(len(ds) for ds in hgr1_ds_list)}')
print(f'Number of samples in FER2013 dataset: {sum(len(ds) for ds in fer2013_ds_list)}')

dataset_df = pd.DataFrame(columns=['dataset', 'num_labels', 'num_samples', 'image_sizes'])
# Nus2Hands dataset
dataset_df = pd.concat([dataset_df, pd.DataFrame({
    'dataset': ['Nus2Hands'],
    'num_labels': [len(nus_2_hand_sign_ds.labels())],
    'num_samples': [len(nus_2_hand_sign_ds)],
    'image_sizes': [set([img.shape for img, _ in nus_2_hand_sign_ds]) if nus_2_hand_sign_ds else None]
})], ignore_index=True)
# HGR1 dataset
for i, ds in enumerate(hgr1_ds_list, start=1):
		dataset_df = pd.concat([dataset_df, pd.DataFrame({
				'dataset': [f'HGR1_id{i:02d}'],
				'num_labels': [len(ds.labels())],
				'num_samples': [len(ds)],
				'image_sizes': [set([img.shape for img, _ in ds]) if ds else None]
		})], ignore_index=True)
# FER2013 dataset
for i, ds in enumerate(fer2013_ds_list, start=1):
		dataset_df = pd.concat([dataset_df, pd.DataFrame({
				'dataset': [f'FER2013_{["train", "test"][i-1]}'],
				'num_labels': [len(ds.labels())],
				'num_samples': [len(ds)],
				'image_sizes': [set([img.shape for img, _ in ds]) if ds else None]
		})], ignore_index=True)
print(dataset_df)

Number of samples in Nus2Hands dataset: 1990
Number of samples in HGR1 dataset: 899
Number of samples in FER2013 dataset: 35887
          dataset num_labels num_samples  \
0       Nus2Hands         10        1990   
1       HGR1_id01         25          75   
2       HGR1_id02         27          54   
3       HGR1_id03         27          54   
4       HGR1_id04         26          53   
5       HGR1_id05         27          84   
6       HGR1_id06         27          85   
7       HGR1_id07         27          86   
8       HGR1_id08         27          83   
9       HGR1_id09         27          81   
10      HGR1_id10         27          81   
11      HGR1_id11         27          81   
12      HGR1_id12         27          82   
13  FER2013_train          7       28709   
14   FER2013_test          7        7178   

                                          image_sizes  
0                                     {(3, 120, 160)}  
1   {(3, 368, 229), (3, 434, 241), (3, 377, 427), ...  

# Define classification train process

1. Define place where the model is saved

In [4]:
time_str = time.strftime("%Y%m%d_%H%M%S")

2. Define train function

In [5]:
def train_process(dataset, model):
  # define batch_size
  batch_size = 64

  # init train val test ds
  train_val_size = int(0.8 * len(dataset))
  test_size = len(dataset) - train_val_size
  train_ds, test_ds = random_split(dataset, [train_val_size, test_size])

  # define optimizer using Adam and loss function
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  loss_fn = torch.nn.CrossEntropyLoss()

  trainer = Trainer(model, optimizer, loss_fn, random_seed_value=86)
  print('device: ', trainer.device)
  avg_loss, metric = trainer.cross_validate(train_ds, k=5, epochs=10, batch_size=batch_size)
  print('avg_loss: ', avg_loss)

  # score model
  test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)
  model_scored = trainer.score(test_dataloader)
  print(f'model_scored: {model_scored:.4f}, avg_accuracy: {100*(1 - model_scored):.4f}')

  # return model scored, train_avg_lost
  return model_scored, avg_loss

3. execute progress

- define the model

In [6]:
models = [
	torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT),
  torchvision.models.densenet121(weights=torchvision.models.DenseNet121_Weights.DEFAULT),
  torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT)
]

- Define tested datasets

In [7]:
datasets = {
    'gi4e_full': ds.Gi4eDataset(
        './datasets/gi4e',
        transform=transforms.Compose([transforms.ToPILImage(), transforms.Resize((224, 224)), transforms.ToTensor()]),
        is_classification=True),
    'gi4e_raw_eyes': ds.ImageDataset(
        './datasets/gi4e_raw_eyes',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
    'gi4e_detected_eyes': ds.ImageDataset(
        './datasets/gi4e_eyes/20250521_200316',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
}

- Train all defined model on each registered datasets

In [8]:
result_df = pd.DataFrame(columns=['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time'])

for name, dataset in datasets.items():
	for model in models:
		print(f'Running {name} dataset with {model.__class__.__name__}')
		# do the train
		start_time = time.time()
		scored, loss = train_process(dataset, model)
		end_time = time.time()
		total_time = end_time - start_time
		print(f'Finished {name} dataset with {model.__class__.__name__}')
		print('----------------------')

		# save the result
		result_df = pd.concat([result_df, pd.DataFrame({
			'model': [model.__class__.__name__],
			'dataset': [name],
			'avg_loss': [loss],
			'avg_accuracy': [scored],
			'total_time': [total_time]
		})], ignore_index=True)

print('Finished all datasets')

# print the result
print(result_df)


Running gi4e_full dataset with ResNet
device:  cuda
Fold 1/5:
Epoch 1/10, Train Loss: 2.4902, Test Loss: 0.9260, Total Time: 00 hours, 00 minutes, 25 seconds
Epoch 2/10, Train Loss: 0.0288, Test Loss: 0.0039, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 3/10, Train Loss: 0.0017, Test Loss: 0.0004, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 4/10, Train Loss: 0.0004, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 5/10, Train Loss: 0.0003, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 6/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 7/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 8/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 9/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 10/10, Train Loss: 0.0001, Test Loss: 0.0000, Total Time: 00 hours, 

C:\Users\nnvuf\AppData\Local\Temp\ipykernel_11664\454607160.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame({


Epoch 1/10, Train Loss: 1.7670, Test Loss: 1.3506, Total Time: 00 hours, 00 minutes, 25 seconds
Epoch 2/10, Train Loss: 0.0225, Test Loss: 0.0560, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 3/10, Train Loss: 0.0041, Test Loss: 0.0035, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 4/10, Train Loss: 0.0016, Test Loss: 0.0011, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 5/10, Train Loss: 0.0008, Test Loss: 0.0007, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 6/10, Train Loss: 0.0006, Test Loss: 0.0005, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 7/10, Train Loss: 0.0004, Test Loss: 0.0004, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 8/10, Train Loss: 0.0004, Test Loss: 0.0003, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 9/10, Train Loss: 0.0003, Test Loss: 0.0003, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 10/10, Train Loss: 0.0002, Test Loss: 0.0003, Total Time: 00 hours, 00 minutes, 18 seconds
Fold 1/5, Total Test Loss: 0.0003, Fold

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

Fold 1/5, Total Test Loss: 0.3503, Fold accuracy: 64.9747
Fold 2/5:
Epoch 1/10, Train Loss: 0.2483, Test Loss: 0.0858, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 2/10, Train Loss: 0.2233, Test Loss: 0.0732, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 3/10, Train Loss: 0.1206, Test Loss: 0.0030, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 4/10, Train Loss: 0.0652, Test Loss: 0.1164, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 5/10, Train Loss: 0.1415, Test Loss: 0.0036, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 6/10, Train Loss: 0.0729, Test Loss: 0.0043, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 7/10, Train Loss: 0.0727, Test Loss: 0.0146, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 8/10, Train Loss: 0.0859, Test Loss: 0.0018, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 9/10, Train Loss: 0.0739, Test Loss: 0.0485, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 10/10, Train Loss: 0.0513, Test Loss: 0.0309, Total Time: 00 h

...

Fold 1/5, Total Test Loss: 0.3956, Fold accuracy: 60.4436
Fold 2/5:
Epoch 1/10, Train Loss: 0.3687, Test Loss: 0.2777, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 2/10, Train Loss: 0.2524, Test Loss: 0.0931, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 3/10, Train Loss: 0.1912, Test Loss: 0.1627, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 4/10, Train Loss: 0.2415, Test Loss: 0.1107, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 5/10, Train Loss: 0.2006, Test Loss: 0.0914, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 6/10, Train Loss: 0.1469, Test Loss: 0.0608, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 7/10, Train Loss: 0.0968, Test Loss: 0.0961, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 8/10, Train Loss: 0.1291, Test Loss: 0.2662, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 9/10, Train Loss: 0.2456, Test Loss: 0.1658, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 10/10, Train Loss: 0.2060, Test Loss: 0.2929, Total Time: 00 h

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

Fold 2/5, Total Test Loss: 0.6885, Fold accuracy: 70.7065
Fold 3/5:
Epoch 1/10, Train Loss: 0.2358, Test Loss: 0.0169, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 2/10, Train Loss: 0.1179, Test Loss: 0.0386, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 3/10, Train Loss: 0.1726, Test Loss: 0.0206, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 4/10, Train Loss: 0.0749, Test Loss: 0.0184, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 5/10, Train Loss: 0.0524, Test Loss: 0.0378, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 6/10, Train Loss: 0.0667, Test Loss: 0.0136, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 7/10, Train Loss: 0.1625, Test Loss: 0.0697, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 8/10, Train Loss: 0.2452, Test Loss: 0.1293, Total Time: 00 hours, 00 minutes, 17 seconds
Epoch 9/10, Train Loss: 0.1372, Test Loss: 0.0615, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 10/10, Train Loss: 0.0629, Test Loss: 0.0334, Total Time: 00 h

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Fold 5/5, Total Test Loss: 1.0597, Fold accuracy: 69.7629
avg_loss:  0.21193921998441717
model_scored: 0.5712, avg_accuracy: 42.8791
Finished gi4e_raw_eyes dataset with VGG
----------------------
Running gi4e_detected_eyes dataset with ResNet
device:  cuda
Fold 1/5:
Epoch 1/10, Train Loss: 0.1111, Test Loss: 1.0260, Total Time: 00 hours, 00 minutes, 30 seconds
Epoch 2/10, Train Loss: 0.0725, Test Loss: 0.2704, Total Time: 00 hours, 00 minutes, 10 seconds
Epoch 3/10, Train Loss: 0.0648, Test Loss: 0.4469, Total Time: 00 hours, 00 minutes, 11 seconds
Epoch 4/10, Train Loss: 0.0316, Test Loss: 0.0501, Total Time: 00 hours, 00 minutes, 10 seconds
Epoch 5/10, Train Loss: 0.0177, Test Loss: 0.0530, Total Time: 00 hours, 00 minutes, 10 seconds
Epoch 6/10, Train Loss: 0.0027, Test Loss: 0.0114, Total Time: 00 hours, 00 minutes, 10 seconds
Epoch 7/10, Train Loss: 0.0007, Test Loss: 0.0010, Total Time: 00 hours, 00 minutes, 10 seconds
Epoch 8/10, Train Loss: 0.0002, Test Loss: 0.0009, Total Time

4. print the result

In [9]:
# swap the first two columns
result_df = result_df[['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time']]
# scale the avg_accuracy to 0-100
result_df['avg_accuracy'] = 100 * (1 - result_df['avg_accuracy'])
# display the total time in the format HH:MM:SS
result_df['total_time'] = pd.to_timedelta(result_df['total_time'], unit='s')

# save the result to csv
result_df.to_csv(f'results_{time_str}.csv', index=False)
# print the result
print(result_df)


              dataset     model  avg_loss  avg_accuracy  \
0           gi4e_full    ResNet  0.000007     99.999652   
1           gi4e_full  DenseNet  0.000076     99.985816   
2           gi4e_full       VGG  0.076536     90.054617   
3       gi4e_raw_eyes    ResNet  0.143681     97.082782   
4       gi4e_raw_eyes  DenseNet  0.003774     98.063464   
5       gi4e_raw_eyes       VGG  0.211939     42.879109   
6  gi4e_detected_eyes    ResNet  0.000156     99.962893   
7  gi4e_detected_eyes  DenseNet  0.000208     99.947998   
8  gi4e_detected_eyes       VGG  0.083605     94.961119   

                 total_time  
0 0 days 00:15:44.519444466  
1 0 days 00:16:33.076972723  
2 0 days 00:20:02.580577135  
3 0 days 00:09:38.321853161  
4 0 days 00:10:28.439326763  
5 0 days 00:15:59.469455004  
6 0 days 00:09:34.802286625  
7 0 days 00:10:18.782304764  
8 0 days 00:15:24.494691133  


In [10]:
print(result_df)

              dataset     model  avg_loss  avg_accuracy  \
0           gi4e_full    ResNet  0.000007     99.999652   
1           gi4e_full  DenseNet  0.000076     99.985816   
2           gi4e_full       VGG  0.076536     90.054617   
3       gi4e_raw_eyes    ResNet  0.143681     97.082782   
4       gi4e_raw_eyes  DenseNet  0.003774     98.063464   
5       gi4e_raw_eyes       VGG  0.211939     42.879109   
6  gi4e_detected_eyes    ResNet  0.000156     99.962893   
7  gi4e_detected_eyes  DenseNet  0.000208     99.947998   
8  gi4e_detected_eyes       VGG  0.083605     94.961119   

                 total_time  
0 0 days 00:15:44.519444466  
1 0 days 00:16:33.076972723  
2 0 days 00:20:02.580577135  
3 0 days 00:09:38.321853161  
4 0 days 00:10:28.439326763  
5 0 days 00:15:59.469455004  
6 0 days 00:09:34.802286625  
7 0 days 00:10:18.782304764  
8 0 days 00:15:24.494691133  


| No |            dataset |    model | avg_loss | avg_accuracy |         total_time |
|---:|-------------------:|---------:|---------:|-------------:|-------------------:|
|  0 |          gi4e_full |   ResNet | 0.000017 |    99.999863 | 00:16:23.117301226 | 
|  1 |          gi4e_full | DenseNet | 0.000099 |    99.996225 | 00:20:05.628188372 | 
|  2 |          gi4e_full |      VGG | 0.030500 |    96.777968 | 00:19:24.024507523 | 
|  3 |      gi4e_raw_eyes |   ResNet | 0.017712 |    99.769400 | 00:08:47.432441950 | 
|  4 |      gi4e_raw_eyes | DenseNet | 0.007763 |    99.778149 | 00:09:55.128549576 | 
|  5 |      gi4e_raw_eyes |      VGG | 0.054540 |    81.848047 | 00:15:27.017767429 | 
|  6 | gi4e_detected_eyes |   ResNet | 0.006875 |    99.928385 | 00:08:44.347270250 | 
|  7 | gi4e_detected_eyes | DenseNet | 0.000107 |    99.968702 | 00:09:53.161640406 | 
|  8 | gi4e_detected_eyes |      VGG | 0.017090 |    85.945651 | 00:15:16.971753836 | 


# Embedded Classification

- Define the model

In [12]:
embedded_models = [md.FeatureExtractor(model) for model in models]
classifier_models = [md.Classifier(model, 103) for model in embedded_models]


- Load the trained weight to the model

In [13]:
# ignore

- Transform the dataset to the embedded data

In [14]:
classifier_df = pd.DataFrame(columns=['key', 'dataset', 'model'])

# Loop through datasets and models to create the classifier_df
print('Starting to get features for datasets...')
for name, dataset in datasets.items():
	for model in embedded_models:
		print(f'Getting features for {name} dataset with {model._get_name()}')

		# get the classifier model from the model
		classifier_model = next((m for m in classifier_models if m.backbone._get_name() == model._get_name()), None)
		if classifier_model is None:
			print(f'Classifier model not found for {model._get_name()}')
			continue

		model_dataset = ds.EmbeddedDataset(dataset, model)

		classifier_df = pd.concat([classifier_df, pd.DataFrame({
				'key': [f'{name}_{model._get_name()}'],
				'model': [classifier_model],
				'dataset': [model_dataset]
		})], ignore_index=True)

		print(f'Finished getting features for {name} dataset with {model._get_name()}')


Starting to get features for datasets...
Getting features for gi4e_full dataset with FeatureExtractor(ResNet)


<All keys matched successfully>


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

Finished training all models

| No|        dataset|                                   model|  avg_loss|  avg_accuracy|  total_time|
|---|---------------|----------------------------------------|----------|--------------|------------|
| 1 |EmbeddedDataset|    Classifier(FeatureExtractor(ResNet))|  0.005883|     99.818135|    4.223327|
| 2 |EmbeddedDataset|  Classifier(FeatureExtractor(DenseNet))|  0.012252|     99.677733|   16.936388|
| 3 |EmbeddedDataset|       Classifier(FeatureExtractor(VGG))|  0.002937|     98.017509|  155.980501|
| 4 |EmbeddedDataset|    Classifier(FeatureExtractor(ResNet))|  0.212613|     95.163104|   16.447177|
| 5 |EmbeddedDataset|  Classifier(FeatureExtractor(DenseNet))|  0.221752|     94.722049|    5.249017|
| 6 |EmbeddedDataset|       Classifier(FeatureExtractor(VGG))|  0.033393|     96.928095|  145.743802|
| 7 |EmbeddedDataset|    Classifier(FeatureExtractor(ResNet))|  0.057876|     97.631673|   18.927852|
| 8 |EmbeddedDataset|  Classifier(FeatureExtractor(DenseNet))|  0.024092|     98.713500|    6.274447|
| 9 |EmbeddedDataset|       Classifier(FeatureExtractor(VGG))|  0.017790|     96.159436|  137.484042|

- Train all defined model on each registered datasets

In [ ]:
result_df = pd.DataFrame(columns=['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time'])

for index, row in classifier_df.iterrows():
	dataset = row['dataset']
	model = row['model']
	key = row['key']

	print(f'Running {key} dataset with {model._get_name()}')
	# do the train
	start_time = time.time()
	scored, loss = train_process(dataset, model)
	end_time = time.time()
	total_time = end_time - start_time
	print(f'Finished {key} dataset with {model._get_name()}')
	print('----------------------')

	# save the result
	result_df = pd.concat([result_df, pd.DataFrame({
			'model': [model._get_name()],
			'dataset': [dataset.__class__.__name__],
			'avg_loss': [loss],
			'avg_accuracy': [scored],
			'total_time': [total_time]
	})], ignore_index=True)
print('Finished training all models')


- Print the result

In [ ]:
# swap the first two columns
result_df = result_df[['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time']]
# scale the avg_accuracy to 0-100
result_df['avg_accuracy'] = 100 * (1 - result_df['avg_accuracy'])
# display the total time in the format HH:MM:SS
result_df['total_time'] = pd.to_timedelta(result_df['total_time'], unit='s')

# save the result to csv
result_df.to_csv(f'results_{time_str}.csv', index=False)
# print the result
print(result_df)


In [ ]:
models = [
	torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT),
  torchvision.models.densenet121(weights=torchvision.models.DenseNet121_Weights.DEFAULT),
  torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT)
]

for model in models:
  # explore the model
	print(f'Model: {model.__class__.__name__}')
	
	# explain the model
	print(f'Number of layers: {len(list(model.children()))}')
	print(model)